In [55]:
import pandas as pd
import seaborn as sns
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
sns.set_theme(style="ticks")

#https://github.com/qpwo/pySCF
from social_choice.profile import Profile#, ballot_box, plurality

#https://github.com/djcunningham0/rankaggregation
import rankaggregation as ra

# Read Submission

In [72]:
base_path      = '/media/workspace/triplet_session/output/evaluation/MLEvaluationTask/results/'

# Real
# model  = "SupervisedModelTraining____mars_gym_model_b____6ce3c531e5_c794172206_sub_False"
# model1 = "SupervisedModelTraining____mars_gym_model_b____b4197a226b_4f1ce3789b_sub_False"
# model2 = "SupervisedModelTraining____mars_gym_model_b____1da67a8f8e_7acd8435e1_sub_False"
# model3 = "SupervisedModelTraining____mars_gym_model_b____c179ab54fa_e752b84f46_sub_False"

# Local

model  = "SupervisedModelTraining____mars_gym_model_b____6ce3c531e5_f5d73b2bdd_sub_True"
model1 = "SupervisedModelTraining____mars_gym_model_b____b4197a226b_c8e7045af6_sub_True"
model2 = "SupervisedModelTraining____mars_gym_model_b____1da67a8f8e_037c9a9f69_sub_True"
model3 = "SupervisedModelTraining____mars_gym_model_b____c179ab54fa_3ad3460c81_sub_True"


files          = [model, model1, model2, model3]
model_names    = ['model', 'model1', 'model2','model3']

In [73]:

df_sub = {}
for i in range(len(files)):
    path = base_path+files[i]+'/submission_{}.csv'.format(files[i].split("/")[-1])
    _df   = pd.read_csv(path, header=None)
    df_sub[model_names[i]] = _df

df_sub.keys()

dict_keys(['model', 'model1', 'model2', 'model3'])

In [74]:
df_sub['model1'].head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1615991,1361154,1748830,400496,473325,455802,919498,1522537,63024,1647194,...,348790,659941,1765999,224659,2050215,578820,2097450,1137769,422938,590374
1,228737,1230986,1997785,1148280,911312,691859,1945218,615390,96806,1014949,...,1111753,1890333,1749576,1135425,1305289,21138,1530283,112200,1025545,1970229
2,17827,1880554,1587422,2041622,158235,1332849,548905,358952,10243,1843848,...,1431833,2068577,1695970,1680032,1157295,536980,1845592,548958,380394,351968
3,21739,2090871,255638,716822,429798,1075129,849156,940123,842438,1787501,...,595136,1885055,537022,155343,474383,1471401,1352725,363435,516485,1645101
4,1314713,1492333,86082,2049207,703524,1554487,176811,1980402,1947925,1095121,...,501011,1123415,510949,18857,1816835,622737,1995244,771409,1238994,332101


In [75]:
df_sub.keys()

dict_keys(['model', 'model1', 'model2', 'model3'])

## Ranked

In [76]:
# import itertools
# import numpy as np
# from collections import OrderedDict

# # list(df_sub.keys())
# def get_list(df_sub, uid, models = []):
#     return list(OrderedDict.fromkeys(list(itertools.chain(*[list(df_sub[k].iloc[uid].values) 
#                                                  for k in models]))))

# candidates = get_list(df_sub, 0, ['model', 'coocorrence', 'most_popular'])
# candidates

In [77]:
def get_rerank(reclist, type = 'instant_runoff'):
    if type == 'instant_runoff':
        return agg.instant_runoff(ballots)
    elif type == 'borda':
        l = agg.borda(ballots)
    elif type == 'dowdall':
        l = agg.dowdall(ballots)
    elif type == 'average_rank':
        l = agg.average_rank(ballots)
    
    return [i[0] for i in l]

agg = ra.RankAggregator()


In [78]:

methods = ['dowdall', 'borda',  'average_rank', 'instant_runoff']
reclist_size = 100

for kind_rerank in methods:

    reranked_list = []
    for uid in tqdm(range(len(df_sub['model1']))):
        ballots  = [list(df_sub[k].iloc[uid].astype(str).values) for k in model_names]
        reranked = list(np.array(get_rerank(ballots, kind_rerank)).astype(int))[:reclist_size]
        reranked_list.append(reranked)

    np.savetxt('data/final_submission_{}.csv'.format(kind_rerank), reranked_list, fmt='%i', delimiter=',') 

100%|██████████| 1000/1000 [01:44<00:00,  9.61it/s]
